In [1]:
from torch import nn
from transformers import AutoTokenizer, AutoModel


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
        self.herbert = AutoModel.from_pretrained("allegro/herbert-base-cased")

        for param in self.herbert.parameters():
            param.requires_grad = False
        

        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(768, 1)
    
    def forward(self, embeddings):
        output = self.herbert(**embeddings)
        output = output['pooler_output']
        output = self.dropout(output)
        output = self.linear(output)
        return nn.Sigmoid()(output)
    

In [2]:
import torch
from torch.utils.data import Dataset, DataLoader

class PolishHateSpeechDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts.loc[index].values[0]
        label = self.labels.loc[index].values[0]

        embbedings = self.tokenizer(
            text,
            padding='max_length',
            add_special_tokens=True,
            return_tensors="pt"
        )
        
        label = torch.tensor(label, dtype=torch.float)


        return embbedings, label


In [3]:
from utils import load_data
from torch.utils.data import DataLoader
import torch


batch_size = 32
num_epochs = 10
lr = 2e-5


model = Model()
tokenizer = model.tokenizer

compute_loss = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)




Some weights of the model checkpoint at allegro/herbert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.sso.sso_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.sso.sso_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [8]:
X_train, X_val, y_train, y_val = load_data()
train_dataset = PolishHateSpeechDataset(X_train, y_train, tokenizer)
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

val_dataset = PolishHateSpeechDataset(X_val, y_val, tokenizer)
val_data_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [9]:
from tqdm import tqdm


def train(model, train_data_loader, val_data_loader, compute_loss, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        for batch_idx, (data, targets) in tqdm(enumerate(train_data_loader), f"Epoch {epoch+1}/{num_epochs}"):
            data['input_ids'] = data['input_ids'].squeeze(1).to(device)
            data['attention_mask'] = data['attention_mask'].squeeze(1).to(device)
            data['token_type_ids'] = data['token_type_ids'].squeeze(1).to(device)
            # print(data)
            # data = data.to(device)
            targets = targets.to(device)
            model.train()
            optimizer.zero_grad()
            

            
        

            outputs = model(data)

            loss = compute_loss(outputs, targets.view(-1, 1).to(torch.float32))

            loss.backward()
            optimizer.step()

            if batch_idx % 5 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}], Batch [{batch_idx+1}/{len(train_data_loader)}], Loss: {loss.item()}")

        # val_loss = evaluate(model, val_data_loader)

        # print(f"Epoch [{epoch+1}/{num_epochs}], Validation Loss: {val_loss}")

    return model

In [10]:
model = model.to(device)

In [11]:
train(model, train_data_loader, val_data_loader, compute_loss, optimizer, num_epochs, device)

Epoch 1/10: 2it [00:08,  3.51s/it]

Epoch [1/10], Batch [1/251], Loss: 0.49637818336486816


Epoch 1/10: 6it [00:23,  4.00s/it]

Epoch [1/10], Batch [6/251], Loss: 0.4804752767086029


Epoch 1/10: 11it [00:37,  3.68s/it]

...

Epoch [1/10], Batch [216/251], Loss: 0.28262948989868164


Epoch 1/10: 221it [11:29,  3.82s/it]

Epoch [1/10], Batch [221/251], Loss: 0.17160829901695251


Epoch 1/10: 223it [11:36,  3.12s/it]


KeyboardInterrupt: 

In [12]:
import torch
from sklearn.metrics import accuracy_score

def evaluate(model, data_loader, compute_loss):
    model.eval()

    total_correct = 0
    total_loss = 0.0
    total_samples = 0

    with torch.no_grad():
        for data, targets in data_loader:
            

            outputs = model(data)
            loss = compute_loss(outputs, targets)

            predicted = torch.round(outputs)
            total_correct += (predicted == targets).sum().item()

            total_loss += loss.item() * data.size(0)
            total_samples += data.size(0)

    average_loss = total_loss / total_samples
    accuracy = total_correct / total_samples

    return average_loss, accuracy


In [13]:
evaluate(model, val_data_loader, compute_loss)

ValueError: too many values to unpack (expected 2)